### Installations

In [1]:
#pip install -q -U bitsandbytes wandb
#pip install -q -U git+https://github.com/huggingface/transformers.git
#pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
#check GPU config
!nvidia-smi

Sun Mar 10 11:00:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import userdata
# Defined in the secrets tab in Google Colab (key icon)
hf_token = userdata.get('HF_TOKEN')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, TextStreamer

: 

### Load model

In [5]:
#config load model in 4bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model_id = "jpacifico/French-Alpaca-7B-Instruct-beta"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, padding_side='left')
streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

### Function to test model

In [7]:
#parameters
temperature: float = 0.7
top_p: float = 1.0
top_k: float = 0
repetition_penalty: float = 1.1
max_new_tokens: int = 1024

In [8]:
def chat_with_frenchalpaca(query: str):
    input_ids = tokenizer.apply_chat_template([{"role": "user", "content": query}], return_tensors="pt").to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        ),
        streamer=streamer,
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_text

In [14]:
prompt = "Donne moi une recette populaire à Aix-En-Provence"
result = chat_with_frenchalpaca(prompt)

Une recette populaire à Aix-en-Provence est la Bouillabaisse, un plat traditionnel de Provence fait avec des poissons blancs et des légumes marins. Voici comment le préparer :

Ingrédients :
- 1 kg de poissons blancs (merlu, cabrol, etc.)
- 2 oignons hachés
- 3 tomates pleines et écrasées
- 2 gousses d'ail écrasées
- 40g de persil frais haché
- 500ml de lait de coco
- Sel et poivre
- Huile d'olive
- Pain pour manger

Préparation :
1. Faire mariner les poissons dans du vin blanc et du sel, puis les placer dans une casserole avec l'huile d'olive. Faire cuire doucement jusqu'à ce qu'ils soient cuits.
2. Dans une autre casserole, faire revenir les oignons, les tomates et l'ail dans de l'huile d'olive. Ajouter le lait de coco et laisser mijoter pendant quelques minutes.
3. Ajouter le persil et les ingrédients de la casserole contenant les poissons. Laisser cuire pendant 10 minutes.
4. Servir la bouillabaisse avec du pain pour manger pour absorber le bouillon.

Astuce : Pour le plus de goût,